In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os # Import the os module

data_dir = '/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions'
files = os.listdir(data_dir) # Now os.listdir should work

In [4]:
paths = [os.path.join(data_dir, file) for file in files if file.endswith('.ipynb')]
names = [file.split("_")[0] for file in files if file.endswith('.ipynb')]

In [5]:
keys = [['kia', 'bmw'], [53727, 505084], 'on', [0.13, 0],
        [33.16, 0], [95.70, 0], [13459, 11295], [85296, 478], -0.67, 15.27]

In [6]:
import os
import io
from nbformat import read
import sys
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

def run_notebook(path):
    nb_name, _ = os.path.splitext(os.path.basename(path))
    dirname = os.path.dirname(path)

    with open(path) as f:
        nb = nbformat.read(f, as_version=4)
    errors = []
    outputs = []
    proc = ExecutePreprocessor(timeout=600, kernel_name='python3')
    proc.allow_errors = True
    for cell in nb.cells:
      if cell["cell_type"] == "code":
        if 'outputs' in cell:
          outputs.append(cell['outputs'])
    return outputs

In [7]:
scores = {}
feedbacks = {}
results = {}
for p in paths:
  print(p)
  feedback = []
  result = []
  notebook_r = run_notebook(p)
  for r in notebook_r:
    try:
      feedback.append(r[0]['evalue'])
    except:
      pass
    try:
      result.append(r[0]['data']['text/plain'])
    except:
      pass
  score = 0
  feedbacks[names[paths.index(p)]+'_error'] = feedback
  results[names[paths.index(p)]] = result
  for key in keys:
    if key in result:
      score = score + 10
  scores[names[paths.index(p)]+'_score'] = score

/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/madalahemalathavenkatesuluhemanandachowdary_217836_19297194_Assignment4_mhc.ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/kethinenisaiakashchowdary_214525_19296708_Assignment4 (3).ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/lavulokeshprasad_208272_19278954_Assignment4.ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/kunchalajogendra_207547_19296056_Assignment4 (1).ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/dukujohn_217219_19297614_Assignment4 .ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/njogususan_LATE_208257_19299701_Assignment4 (1).ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions/keniyakaranjayesh_218571_19294092_Assignment4 (1).ipynb
/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submis

In [8]:
import os
import io
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import pandas as pd

# Function to run and extract output from notebooks
def run_notebook(path):
    nb_name, _ = os.path.splitext(os.path.basename(path))
    dirname = os.path.dirname(path)

    with open(path) as f:
        nb = nbformat.read(f, as_version=4)
    errors = []
    outputs = []
    proc = ExecutePreprocessor(timeout=600, kernel_name='python3')
    proc.allow_errors = True
    proc.preprocess(nb, {'metadata': {'path': dirname}})

    # Collecting outputs from cells
    for cell in nb.cells:
        if cell.cell_type == "code":
            for output in cell.outputs:
                if output.output_type == "execute_result" and 'data' in output and 'text/plain' in output.data:
                    outputs.append(output.data['text/plain'])
    return outputs

# Directory and file paths
data_dir = '/content/drive/MyDrive/INFO 570 - Grading/Week 4/INFO 570 - 92/submissions'
files = os.listdir(data_dir)
paths = [os.path.join(data_dir, file) for file in files if file.endswith('.ipynb')]
names = [file.split("_")[0] for file in files if file.endswith('.ipynb')]

# Dictionary to store results and scores
results = {}
scores = {}

# Process each notebook and calculate score
for path, name in zip(paths, names):
    output_texts = run_notebook(path)
    results[name] = output_texts
    score = 0
    # Add 10 to score for each 'pass' output
    for output_text in output_texts:
        if 'pass' in output_text:
            score += 10
    scores[f"{name}_score"] = score

# Prepare DataFrame for final output, including 'name' column
length = [len(results[name]) for name in names]
max_length = max(length)

output = pd.DataFrame(columns=['name'] + [f'output_{i+1}' for i in range(max_length)] + ['score', 'feedback'])
output['score'] = 0
output['feedback'] = 0

# Fill DataFrame with outputs, scores, and names, padding with None if needed
for idx, name in enumerate(names):
    row = [name] + results[name] + [None] * (max_length - len(results[name])) + [scores[f"{name}_score"], 0]  # Initial feedback as 0
    output.loc[idx] = row

# Reset the index to make it a simple range index
output = output.reset_index(drop=True)

# Ensure 'score' column is numeric to avoid any ambiguity
output['score'] = pd.to_numeric(output['score'], errors='coerce')

# Define general feedback text
general_feedback = (
    "Avoid hardcoding solutions; instead, use dynamic approaches like functions for calculations, slicing, and string manipulations. Leverage methods like .lower() or .upper() for case adjustments to ensure consistency. Pay close attention to syntax details, such as using .strip() or .replace() to handle spaces and % for remainder calculations effectively. Be mindful of data types—convert between types as needed, such as using list() to handle sets or int() to manage decimals. These practices help ensure your code is both accurate and adaptable. Finally, carefully follow assignment instructions, particularly for formatting, capitalization, and spacing in strings. These refinements will greatly improve the quality and efficiency of your work."
)

# Iterate over rows using .iloc to ensure position-based access
for i in range(len(output)):
    if pd.notna(output.iloc[i]['score']) and output.iloc[i]['score'] < 100:
        output.at[i, 'feedback'] = str(output.at[i, 'feedback']) + " " + general_feedback
    else:
        output.at[i, 'feedback'] = "Good Job!"

# Display or save the final DataFrame as needed
print(output)

                                           name  \
0   madalahemalathavenkatesuluhemanandachowdary   
1                     kethinenisaiakashchowdary   
2                              lavulokeshprasad   
3                              kunchalajogendra   
4                                      dukujohn   
..                                          ...   
93                               banjaradeepika   
94                             tohfarebekaislam   
95             katagondanahallinagarajuchinmaya   
96                  dobariyahinkalbenpareshbhai   
97                                  ochiengbill   

                                             output_1  \
0   Empty DataFrame\nColumns: [<!DOCTYPE html><htm...   
1   Empty DataFrame\nColumns: [<!DOCTYPE html><htm...   
2   Empty DataFrame\nColumns: [<!DOCTYPE html><htm...   
3      year make     model        trim   body tran...   
4   Empty DataFrame\nColumns: [<!DOCTYPE html><htm...   
..                                           

<ipython-input-8-957ad3c4a04f>:76: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 Avoid hardcoding solutions; instead, use dynamic approaches like functions for calculations, slicing, and string manipulations. Leverage methods like .lower() or .upper() for case adjustments to ensure consistency. Pay close attention to syntax details, such as using .strip() or .replace() to handle spaces and % for remainder calculations effectively. Be mindful of data types—convert between types as needed, such as using list() to handle sets or int() to manage decimals. These practices help ensure your code is both accurate and adaptable. Finally, carefully follow assignment instructions, particularly for formatting, capitalization, and spacing in strings. These refinements will greatly improve the quality and efficiency of your work.' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

In [10]:
output

,name,output_1,output_2,output_3,output_4,output_5,output_6,output_7,output_8,output_9,...,output_15,output_16,output_17,output_18,output_19,output_20,output_21,output_22,score,feedback
0,madalahemalathavenkatesuluhemanandachowdary,Empty DataFrame\nColumns: [<!DOCTYPE html><htm...,{'question 1': 'fail'},"{'question 1': 'fail', 'question 2': 'fail'}","{'question 1': 'fail', 'question 2': 'fail', '...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...",...,None,None,None,None,None,None,None,None,0,"0 Avoid hardcoding solutions; instead, use dyn..."
1,kethinenisaiakashchowdary,Empty DataFrame\nColumns: [<!DOCTYPE html><htm...,{'question 1': 'fail'},"{'question 1': 'fail', 'question 2': 'fail'}","{'question 1': 'fail', 'question 2': 'fail', '...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...",...,None,None,None,None,None,None,None,None,0,"0 Avoid hardcoding solutions; instead, use dyn..."
2,lavulokeshprasad,Empty DataFrame\nColumns: [<!DOCTYPE html><htm...,{'question 1': 'fail'},"{'question 1': 'fail', 'question 2': 'fail'}","{'question 1': 'fail', 'question 2': 'fail', '...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...",...,None,None,None,None,None,None,None,None,0,"0 Avoid hardcoding solutions; instead, use dyn..."
3,kunchalajogendra,year make model trim body tran...,year make model trim body transmission ...,{'question 1': 'pass'},"{'question 1': 'pass', 'question 2': 'pass'}","{'question 1': 'pass', 'question 2': 'pass', '...","{'question 1': 'pass',\n 'question 2': 'pass',...","TtestResult(statistic=33.16074220311518, pvalu...","{'question 1': 'pass',\n 'question 2': 'pass',...","TtestResult(statistic=95.6951806871487, pvalue...",...,"{'question 1': 'pass',\n 'question 2': 'pass',...",None,None,None,None,None,None,None,110,Good Job!
4,dukujohn,Empty DataFrame\nColumns: [<!DOCTYPE html><htm...,{'question 1': 'fail'},"{'question 1': 'fail', 'question 2': 'fail'}","{'question 1': 'fail', 'question 2': 'fail', '...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...","{'question 1': 'fail',\n 'question 2': 'fail',...",...,None,None,None,None,None,None,None,None,0,"0 Avoid hardcoding solutions; instead, use dyn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,banjaradeepika,year make model trim body tran...,year make model trim body transmission ...,{'question 1': 'pass'},"{'question 1': 'pass', 'question 2': 'pass'}","{'question 1': 'pass', 'question 2': 'pass', '...","{'question 1': 'pass',\n 'question 2': 'pass',...","TtestResult(statistic=33.16074220311518, pvalu...","TtestResult(statistic=33.16074220311518, pvalu...","{'question 1': 'pass',\n 'question 2': 'pass',...",...,"{'question 1': 'pass',\n 'question 2': 'pass',...","{'question 1': 'pass',\n 'question 2': 'pass',...","{'question 1': 'pass',\n 'question 2': 'pass',...",None,None,None,None,None,110,Good Job!
94,tohfarebekaislam,year make model trim body tran...,year make model trim body transmission ...,{'question 1': 'pass'},"{'question 1': 'pass', 'question 2': 'pass'}","{'question 1': 'pass', 'question 2': 'pass', '...","{'question 1': 'pass',\n 'question 2': 'pass',...","TtestResult(statistic=33.16074220311518, pvalu...","{'question 1': 'pass',\

In [9]:
from google.colab import files

# Save to Excel, explicitly specifying filetype with '.xlsx' extension
output.to_excel('output.xlsx', index=False)

# Download the file
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>